Sequestration algorithm
Separate an incoming data batch into open and sequestered using
stratified sampling

Determine which version of python notebook relies on

In [37]:
import sys
print(sys.executable) 

/usr/local/Cellar/jupyterlab/3.6.1/libexec/bin/python3.11


Import modules and set variables

In [38]:
# Import modules and set variables
import os
import pandas as pd
import numpy as np
import math
from datetime import date
import random
import copy
cwd = os.getcwd()

Functions

In [39]:
##### Discuss with MIDRC: Custom function, .ipynb vs .py file
# Matlab has a function named "groupcounts". Below is a pythonic recreation:
def groupcounts(df, col_name):
    df_out = df[col_name].value_counts().reset_index(name='GroupCount')
    df1 = df[col_name].value_counts(normalize =True).reset_index(name='Percent')
    df_out['Percent'] = df1['Percent']*100
    df_out = df_out.sort_values(['index'])
    df_out = df_out.rename(columns={'index': col_name}).reset_index(drop=True)
    return df_out

Read in Data

In [40]:
##### Discussion with MIDRC: Promt was removed. File names & paths hard-coded
# Read in Data
# prompt = "Copy filepath to input data: "
# filepath = input(prompt)
# Example from my computer
filepath = "/Users/alecsteep/Documents/matlab2python/data/testing_set/"

#if len(filepath) == 0:
#    filepath = cwd

# prompt = 'Input filename: '
# filename = input(prompt)
# Example from my computer
filename = "sequestration_data_RSNA_20230131.tsv"
#filename = "sequestration_data_RSNA_20230214.tsv"

# Verify fields read in with correct data type
inputfile = filepath + filename
# In the matlab script, the data object is named "data". We'll maintain that nomenclature as a pandas data.frame
data = pd.read_csv(inputfile,sep = '\t')
# Adjust the data types
data['submitter_id'] = data['submitter_id'].astype(str)
data["age_at_index"] = pd.to_numeric(data["age_at_index"])

In [41]:
# Check for duplicates - If warning presents, go to merge batch
ptcount = np.unique(data['submitter_id'])
if len(ptcount) != len(data.index):
    dupes_n = len(data.index)-len(ptcount)
    print("WARNING: " + str(dupes_n) + " duplicate patients in batch \n")

Clean Data

In [42]:
# Clean Data
# sex race ethnicity age_at_index covid19_positive site_id modality
for i in range(0, len(data.index)):
    if data.loc[i,'sex'] == "":
        data.loc[i,'sex'] = 'Not Reported'
    if data.loc[i, 'ethnicity'] == "":
        data.loc[i, 'ethnicity'] = 'Not Reported'
    if data.loc[i, 'race'] == "":
        data.loc[i, 'race'] = 'Not Reported'
    if data.loc[i, 'covid19_positive'] == "":
        data.loc[i, 'covid19_positive'] = 'Not Reported'
    if math.isnan(data.loc[i, 'age_at_index']) or pd.isnull(data.loc[i, 'age_at_index']):
        if data.loc[i, 'age_at_index_gt89'] == "Yes":
            data.loc[i,'age_at_index'] = 890
        else:
            data.loc[i,'age_at_index'] = 999

Convert columns to strings (sex, race, ethnicity, covid19_positive, dataset)

In [43]:
# Convert columns to strings
data['sex'] = data['sex'].astype(str)
data['race'] = data['race'].astype(str)
data['ethnicity'] = data['ethnicity'].astype(str)
data['covid19_positive'] = data['covid19_positive'].astype(str)
data['dataset'] = data['dataset'].astype(str)

Modality

In [44]:
# Modality
M = groupcounts(data, 'modality')

# The modality factors are pre-defined by original author
modalityNames = ["CR","CT","DX","MR"]

# One hot encode modality column with booleans
for mn in modalityNames:
    data[mn] = data.modality.str.contains(mn).astype('uint8')

ModalityCount = [sum(data['CR']), sum(data['CT']), sum(data['DX']), sum(data['MR'])]
    
M1 = pd.DataFrame({
    'modalityNames':modalityNames,
    'ModalityCount':ModalityCount
}).sort_values(['ModalityCount'], ascending=[False])
modalityNames = M1.modalityNames.tolist()

In [45]:
##### Discuss with MIDRC: Natalie's code incorporates batch, this code does not, missing file
# Pre-assign dataset and compare with previous batches
FinalTable = copy.copy(data)
FinalTable['batch'] = "Undefined"

Separate age into CDC groups

In [46]:
##### Discuss with MIDRC: Location of known bug
# Separate age into CDC groups
# CDC COVID data uses two sets of age groups: 
# (1) age-groups consistent with those used across CDC COVID-19 surveillance pages
# (2) age groups that are routinely included in NCHS morality reports
# Assume 0-17, 18-29, 30-39, 40-49, 50-64, 65-74, 75-84, 85+
# From https://www.cdc.gov/nchs/nvss/vsrr/covid_weekly/index.htm#SexAndAge
data['agec'] = 0
for i in range(0, len(data)):
    if data.loc[i, 'age_at_index'] <= 17:
        data.loc[i, 'agec'] = 1
    elif data.loc[i, 'age_at_index'] > 17 and data.loc[i, 'age_at_index'] <= 29:
        data.loc[i, 'agec'] = 2
    elif data.loc[i, 'age_at_index'] > 29 and data.loc[i, 'age_at_index'] <= 39:
        data.loc[i, 'agec'] = 3
    elif data.loc[i, 'age_at_index'] > 39 and data.loc[i, 'age_at_index'] <= 49:
        data.loc[i, 'agec'] = 4
    elif data.loc[i, 'age_at_index'] > 49 and data.loc[i, 'age_at_index'] <= 64:
        data.loc[i, 'agec'] = 5
    elif data.loc[i, 'age_at_index'] > 64 and data.loc[i, 'age_at_index'] <= 74:
        data.loc[i, 'agec'] = 6
    elif data.loc[i, 'age_at_index'] > 74 and data.loc[i, 'age_at_index'] <= 84:
        data.loc[i, 'agec'] = 7
    elif data.loc[i, 'age_at_index'] > 84 and data.loc[i, 'age_at_index'] <= 140:
        data.loc[i, 'agec'] = 8
    elif data.loc[i, 'age_at_index'] == 890:
        data.loc[i, 'agec'] = 8


Grab site information and initialize seed

In [47]:
# Grab site information and initialize seed

#sites = data['site_id'].value_counts().reset_index(name='GroupCount')
#df1 = data['site_id'].value_counts(normalize =True).reset_index(name='Percent')
#sites['Percent'] = df1['Percent']*100
#sites = sites.rename(columns={'index': 'site_id'})
sites = groupcounts(data, 'site_id')

datasave = copy.copy(data)

# Initialize seed as date of sequestration
t = date.today()
# Note: This command returns a similar, but slightly different value to the matlab equivalent
# For notes on datenum conversion: https://stackoverflow.com/questions/32991934/equivalent-function-of-datenumdatestring-of-matlab-in-python
seed = date.toordinal(t)+366
np.random.seed(seed)

Split for each site

In [48]:
##### Discuss with MIDRC: Many nested for loops and conditionals: P(coding error) increases
# Split for each site
percSeq = 0.2

for x in range(0, len(sites)):
    data = copy.copy(datasave.loc[datasave['site_id'] == sites.loc[x,'site_id']])
    
    # Gather stats
    A = groupcounts(data,'agec')
    S = groupcounts(data,'sex')
    R = groupcounts(data,'race')
    E = groupcounts(data,'ethnicity')
    C = groupcounts(data, 'covid19_positive')
    
    # Split 
    # Assumes all unique patients
    # Separate on C, A, R, G, E, and modality
    open1 = pd.DataFrame([])
    seq = pd.DataFrame([])
    i = 0
    feature_list = ['case_n','age_index', 'race_index','sex_index','eth_index']
    count = pd.DataFrame(0, index=np.arange(len(A)*len(S)*len(R)*len(E)*len(C)), columns=feature_list)
    # count = zeros(height(C)*height(A)*height(R)*height(S)*height(E),5);
    for m in range(0,len(modalityNames)):
        modality = modalityNames[m]
        # Select all patients with given modality inclusive of multiples
        tempm = data.loc[data[modality] == 1]
        # Remove these from data such that they are not selected again (unnecessary)
        i_drops = data.loc[data[modality] == 1].index
        data.drop(labels = i_drops, axis = 0, inplace = True)
        for cg in range(0, len(C)):
            covid_group = C.loc[cg,'covid19_positive']
            tempc = tempm.loc[tempm['covid19_positive'] == covid_group]
            for ag in range(-1, len(A)):
                if(ag == -1):
                    age_group = 0
                else:
                    age_group = A.loc[ag,'agec']
                temp1 = tempc.loc[tempc['agec'] == age_group]
                for rg in range(0, len(R)):
                    race_group = R.loc[rg,'race']
                    temp2 = temp1.loc[temp1['race'] == race_group]
                    for sg in range(0, len(S)):
                        sex_group = S.loc[sg,'sex']
                        temp3 = temp2.loc[temp2['sex'] == sex_group]
                        for eg in range(0, len(E)):
                            ethnicity_group = E.loc[eg,'ethnicity']
                            temp4 = temp3.loc[temp3['ethnicity'] == ethnicity_group]
                            count.loc[i,] = [len(temp4),ag,rg,sg,eg]
                            #print(i)
                            i = i + 1
                            if len(temp4) > 0:
                                if len(temp4) < 5:
                                    # Create a list of uniformly distributed random numbers 
                                    # [0,1] the length of the number of cases in that strata
                                    list = np.random.rand(len(temp4),1)
                                    # If rand(n) <=0.2 assign test
                                    # If rand(n) >0.2 assign train
                                    for n in range(0,len(list)):
                                        if list[n] > percSeq:
                                            add1 = pd.DataFrame(temp4.loc[temp4.index[n]]).transpose()
                                            #add1 = pd.DataFrame(temp4.loc[temp4.index[n]])
                                            open1 = pd.concat([open1, add1])
                                            #print('part1:' + str(open1.shape))
                                        else:
                                            add2 = pd.DataFrame(temp4.loc[temp4.index[n]]).transpose()
                                            #add2 = pd.DataFrame(temp4.loc[temp4.index[n]])
                                            seq = pd.concat([seq, add2])
                                else:
                                    rows = len(temp4)
                                    # Shuffle the order of cases in this strata and than assign
                                    # the first 80% to train and the last 20% to test
                                    arr = np.array(range(0, rows))
                                    idx = np.random.permutation(arr)
                                    add1 = pd.DataFrame(temp4.loc[temp4.index[idx[range(0, round(rows*(1-percSeq)))]]])
                                    open1 = pd.concat([open1, add1])
                                    add2 = pd.DataFrame(temp4.loc[temp4.index[idx[range(round(rows*(1-percSeq)),len(idx))]]])
                                    seq = pd.concat([seq, add2])
                                    #print(open1.shape)
                                    
    
    
    # Write results in Final Table
    for i in range(0, len(open1)):
        idx = FinalTable.index[FinalTable['submitter_id'] == open1['submitter_id'].iloc[i]].tolist()
        FinalTable.loc[idx, 'dataset'] = "Open"
        FinalTable.loc[idx, 'batch'] = filename[len(filename)-12:len(filename)-4]

    for i in range(0, len(seq)):
        idx = FinalTable.index[FinalTable['submitter_id'] == seq['submitter_id'].iloc[i]].tolist()
        FinalTable.loc[idx, 'dataset'] = "Seq"
        FinalTable.loc[idx, 'batch'] = filename[len(filename)-12:len(filename)-4]

idx = FinalTable.index[FinalTable['submitter_id'] == "Unassigned"].tolist()
if len(idx) > 0:
    print("Warning: " + str(len(idx)) +" patients did not fall in sequestration criteria \n")
    print("Assigning to open dataset \n")
    FinalTable.loc[idx, 'dataset'] = "Open"
    FinalTable.loc[idx, 'batch'] = filename[len(filename)-12:len(filename)-4]


Check for duplicate patients

In [49]:
##### Discuss with MIDRC: Bottle neck, searches fer duplicates could be made more efficient
# Check for duplicate patients
ptcount = pd.unique(FinalTable['submitter_id'])
if not len(ptcount) == len(FinalTable):
    dupes = len(FinalTable) - len(ptcount)
    print("WARNING: "+str(dupes)+" duplicate patients in batch \n")
    count = pd.DataFrame(0, index=np.arange(len(FinalTable)), columns=['duped'])
    for i in range(0, len(FinalTable)):
        for j in range(0, len(FinalTable)):
            if FinalTable.loc[i,'submitter_id'] == FinalTable.loc[j,'submitter_id']:
                count.loc[i, 'duped'] = count.loc[i, 'duped'] + 1
    dupidx = count.index[count['duped'] > 1].tolist()
    datadup = FinalTable.loc[dupidx]
    datadup


Write Gen3 file

In [50]:
##### Discussion with MIDRC: Prompt was removed
#prompt = "Copy filepath to output data: "
#filepath = input(prompt)
if len(filepath) == 0:
    filepath = cwd
# Example from my computer
# filepath = "/Users/alecsteep/Documents/matlab2python/data/testing_set/"
filepath = str(filepath)
#FinalTable = removevars(FinalTable, 'study_description');
#Gen3Table = [FinalTable(:,1:12) FinalTable(:,17)];
# Sort the final table
FinalTable = FinalTable.sort_values(['submitter_id'], ascending=[True]).reset_index(drop=True)
Gen3Table = FinalTable
file_name = filepath + "Gen3_Files/" + "COMPLETED_" + filename[0:len(filename)-4] + ".tsv"
Gen3Table.to_csv(file_name, sep='\t', encoding='utf-8', index=False)

In [51]:
#count.loc[i, 'duped'] = count.loc[i, 'duped'] + 1
#count.loc[i, 'duped']

#.reset_index(drop=True)
#C = np.unique(FinalTable['submitter_id'])
#C
#ia = range(0,len(C))
#ia
input_file = copy.copy(filename)
total_count = len(FinalTable)
open_count = len(FinalTable[FinalTable['dataset'] == "Open"])
seq_count = len(FinalTable[FinalTable['dataset'] == "Seq"])
site_list = np.unique(sites['site_id'])
log = pd.DataFrame([t, input_file,seed,total_count,open_count,seq_count,site_list]).transpose()
log.columns =["t","input_file","seed","total_count","open_count","seq_count","site_list"]
log
if not os.path.exists(filepath + "MIDRC_EXAMPLE_sequestration_log.csv"):
    log.to_csv(filepath + "MIDRC_EXAMPLE_sequestration_log.tsv", sep='\t', encoding='utf-8', index=False)
else:
    log.to_csv(filepath + "MIDRC_EXAMPLE_sequestration_log.tsv", sep='\t', encoding='utf-8', index=False, mode='a', header = False)

Write entry in seq log

In [52]:
# Write entry in seq log
FinalTable = FinalTable[FinalTable['batch'] == filename[len(filename)-12:len(filename)-4]]
FinalTable[FinalTable['submitter_id'] == np.unique(FinalTable['submitter_id'])].reset_index(drop=True)
C = np.unique(FinalTable['submitter_id'])
ia = range(0,len(C))

# date, filename, seed, sites, total count, open count, seq count
input_file = copy.copy(filename)
total_count = len(FinalTable)
open_count = len(FinalTable[FinalTable['dataset'] == "Open"])
seq_count = len(FinalTable[FinalTable['dataset'] == "Seq"])
site_list = np.unique(sites['site_id'])
log = pd.DataFrame([t, input_file,seed,total_count,open_count,seq_count,site_list]).transpose()
log.columns =["t","input_file","seed","total_count","open_count","seq_count","site_list"]
log
if not os.path.exists(filepath + "MIDRC_EXAMPLE_sequestration_log.csv"):
    log.to_csv(filepath + "MIDRC_EXAMPLE_sequestration_log.tsv", sep='\t', encoding='utf-8', index=False)
else:
    log.to_csv(filepath + "MIDRC_EXAMPLE_sequestration_log.tsv", sep='\t', encoding='utf-8', index=False, mode='a', header = False)

In [ ]:
# This last bit of code is matlab. Original author notes in video that this code is not necessary and it's 
# function is to create pi charts for visual checks. This code was not converted to python, it was inferred that
# is was not desired for translation. However, code may be translated upon request.


#%% Write results to data evaluation sheet
#% One per batch

#% Filename "MIDRC_sequestration_data_<recieved date>_<site_id>_<date ran>.xlsx"
#evalsheet = filepath + "Eval_" + filename(1:end-5) +".xlsx";
#copyfile(filepath + "/MIDRC_sequestration_site-id_MM_DD_YY.xlsx", evalsheet)
    
#InputTable = FinalTable;
#[A,R,S,E,C,M] = CountSeqCategories(InputTable);
#writematrix(A,evalsheet,'Sheet',2,'Range','B5:B13')
#writematrix(R,evalsheet,'Sheet',2,'Range','B15:B21')
#writematrix(S,evalsheet,'Sheet',2,'Range','B23:B26')
#writematrix(E,evalsheet,'Sheet',2,'Range','B28:B30')
#writematrix(C,evalsheet,'Sheet',2,'Range','B32:B34')
#writematrix(M,evalsheet,'Sheet',2,'Range','B36:B39')
    
#open = InputTable(InputTable.dataset == "Open",:);
#[A,R,S,E,C,M] = CountSeqCategories(open);
#writematrix(A,evalsheet,'Sheet',2,'Range','F5:F13')
#writematrix(R,evalsheet,'Sheet',2,'Range','F15:F21')
#writematrix(S,evalsheet,'Sheet',2,'Range','F23:F26')
#writematrix(E,evalsheet,'Sheet',2,'Range','F28:F30')
#writematrix(C,evalsheet,'Sheet',2,'Range','F32:F34')
#writematrix(M,evalsheet,'Sheet',2,'Range','F36:F39')
    
#seq = InputTable(InputTable.dataset == "Seq",:);
#[A,R,S,E,C,M] = CountSeqCategories(seq);
#writematrix(A,evalsheet,'Sheet',2,'Range','J5:J13')
#writematrix(R,evalsheet,'Sheet',2,'Range','J15:J21')
#writematrix(S,evalsheet,'Sheet',2,'Range','J23:J26')
#writematrix(E,evalsheet,'Sheet',2,'Range','J28:J30')
#writematrix(C,evalsheet,'Sheet',2,'Range','J32:J34')
#writematrix(M,evalsheet,'Sheet',2,'Range','J36:J39')